# Fraud XGB

In [ ]:
# https://www.kaggle.com/arnocandel/python-datatable
# more information: http://github.com/h2oai/datatable
!pip install --upgrade https://s3.amazonaws.com/artifacts.h2o.ai/releases/ai/h2o/pydatatable/0.8.0.dev115/x86_64-centos7/datatable-0.8.0.dev115-cp36-cp36m-linux_x86_64.whl
    
# Latest Pandas version
!pip install -q 'pandas==0.25' --force-reinstall

In [ ]:
import datatable as dt
import pandas as pd
print("DataTable version:", dt.__version__)
print("Pandas version:", pd.__version__)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import time
notebookstart = time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from contextlib import contextmanager
import gc; gc.enable()
import pprint
import time

import datetime
import csv
import random

from sklearn import preprocessing

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.model_selection import KFold, train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve

import eli5
import xgboost as xgb
print("XGB version:", xgb.__version__)

seed = 24
np.random.seed(seed)

pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 500)

In [ ]:
print("Drop Cols..")
important_cols = ['R_emaildomain_suffix','V1','V10','V100','V101',
'V102','V103','V104','V105','V106','V108','V109','V11','V110','V111','V112','V113','V114','V115','V116','V118','V12','V120',
'V121','V122','V123','V124','V125','V13','V135','V137','V139','V140','V142','V143','V144','V145','V146','V148','V149','V150',
'V151','V152','V153','V154','V155','V156','V157','V158','V159','V160','V161','V162','V164','V165','V166','V167','V168','V169',
'V17','V170','V171','V172','V173','V174','V175','V176','V177','V178','V179','V18','V180','V182','V183','V184','V185','V187',
'V188','V189','V190','V191','V192','V194','V195','V199','V2','V200','V201','V202','V203','V204','V205','V206','V207','V208',
'V21','V210','V211','V212','V213','V214','V215','V216','V217','V218','V219','V220','V221','V222','V223','V224','V225','V226',
'V227','V228','V229','V230','V231','V232','V233','V234','V236','V237','V238','V239','V24','V242','V243','V244','V245','V246',
'V247','V248','V249','V25','V250','V251','V252','V253','V254','V255','V256','V257','V259','V26','V260','V261','V262','V263',
'V264','V265','V266','V267','V268','V270','V271','V272','V273','V274','V275','V276','V277','V278','V279','V280','V284','V286',
'V287','V288','V289','V29','V290','V291','V292','V293','V295','V297','V298','V299','V3','V30','V300','V301''V303','V304',
'V319','V32','V322','V323','V324','V326','V327','V33','V331','V332','V333','V335','V336','V337','V338','V339','V34','V37',
'V39','V4','V40','V41','V42','V43','V44','V46','V48','V49','V50','V51''V52','V55','V57','V58','V59','V6','V60','V63','V64',
'V66','V67','V69','V7','V70','V71','V72','V73','V74','V75','V76','V77','V79','V8','V80','V81','V84','V85','V9','V90','V91',
'V93','V94','V95','V97','V98',
'distgroup_0_count','distgroup_nan_sum','id_03','id_04','id_07','id_08','id_09','id_10','id_11','id_12','id_15','id_16',
'id_17','id_21','id_22','id_23','id_25','id_26','id_28','id_29','id_32','id_34','id_35','id_36','id_37','id_38',
'idgroup_nan_sum']

drop_more = ['C8_AMT_12h_mean', 'card4_AMT_5d_sum', 'idgroup_mean',
       'idgroup_sum', 'C8_AMT_5d_count', 'C8_AMT_12h_count',
       'C1_AMT_12h_mean', 'C7_AMT_12h_sum', 'C8_AMT_5d_mean',
       'C3_AMT_5d_mean', 'C6_AMT_5d_count', 'C1_AMT_12h_count',
       'C7_AMT_5d_count', 'id_02_to_std_card4', 'C12_AMT_5d_count',
       'C14_AMT_12h_sum', 'card4_AMT_5d_count', 'C10_AMT_5d_mean',
       'card6_AMT_12h_count', 'C11_AMT_5d_count', 'Transactiongroup_sum',
       'card6_AMT_12h_sum', 'D5', 'DeviceInfo', 'id_02', 'C8_AMT_12h_sum',
       'C6_AMT_12h_sum', 'C12', 'id_02_to_std_card1', 'Dgroup_nan_sum',
       'C4_AMT_5d_sum', 'C7_AMT_12h_mean', 'card2_AMT_12h_mean', 'V45',
       'V83', 'id_02_to_mean_card4', 'V96', 'V328', 'C11_AMT_12h_sum',
       'C9_AMT_12h_mean', 'V78', 'C4_AMT_12h_mean', 'V119',
       'C3_AMT_12h_mean', 'V129', 'V283', 'card3_AMT_5d_count', 'D9',
       'V134']

drop = ['level_0', 'index', 'month', 'year',
        'TransactionDT', 'day', 'isFraud2'] + important_cols  + drop_more

In [ ]:
folder_path1 = '../input/ieee-fraud-detection/'
folder_path2 = '../input/ieee-fraud-feature-engineering-and-eda/'



def reduce_mem_usage(df, verbose=True):
    print("\nMemeory Usage Before:")
    print(df.info(memory_usage = 'deep'))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    
    print("Memeory Usage After:")
    print(df.info(memory_usage = 'deep'))
    return df

@contextmanager
def timer(name):
    """
    Time Each Process
    """
    t0 = time.time()
    yield
    print('\n[{}] done in {} Minutes'.format(name, round((time.time() - t0)/60,2)))

def fe_read(table):        
    with timer("Read {}".format(table)):
        df_transaction = dt.fread(f'{folder_path1}{table}_transaction.csv')
        df_identity = dt.fread(f'{folder_path1}{table}_identity.csv')
        df_identity.key = 'TransactionID'
        df = df_transaction[:, :, dt.join(df_identity)]
        df_fe = dt.fread(f'{folder_path2}{table}_fraud_fe_nb.csv')
        df_fe.names = {'isFraud': 'isFraud2'}
        df_fe.key = 'TransactionID'
        df_out = df[:, :, dt.join(df_fe)]
        
        keepcols = [x for x in df_out.names if x not in drop]
        
    return df_out[:,keepcols]

def fraud_process(important_cols):
    with timer("Fraud Pre-Process"):
        with timer("Concat"):            
            tr_df = fe_read(table= 'train').to_pandas()
            te_df = fe_read(table= 'test').to_pandas()
            
            df = pd.concat([tr_df, te_df],
                           axis = 0, sort = True).reset_index()
            df.set_index('TransactionID', inplace=True)
            print(df.shape)

        with timer("Label Encode"):
            categorical_cols = []
            # Label Encoding
            for f in [x for x in df.columns if x is not 'isFraud']:
                if df[f].dtype=='object': 
                    categorical_cols += [f]
                    lbl = preprocessing.LabelEncoder()
                    df[f] = lbl.fit_transform(df[f].fillna(-9).astype(str))

        with timer("Reduce Memory"):
            df = reduce_mem_usage(df)

        df.loc[df.isFraud == 0, 'isFraud'] = np.nan
        print(df.shape)
        print(df.isFraud.value_counts())

        train = df.loc[df.isFraud.notnull(), :].fillna(-9)
        y = df.loc[df.isFraud.notnull(), 'isFraud'].copy() - 1
        test = df.loc[df.isFraud.isnull(), :].fillna(-9)

        train.drop('isFraud', axis = 1, inplace=True)
        test.drop('isFraud', axis = 1, inplace=True)

        feature_subset = train.columns

        split_size = 0.25
        X_train, X_valid, y_train, y_valid = train_test_split(
            train[feature_subset], y, test_size=split_size,
            random_state=seed, shuffle=False)
        del train; gc.collect()
        # XGBOOST Efficient Feature Storage
        d_train = xgb.DMatrix(X_train, y_train,feature_names=feature_subset)
        del X_train ;gc.collect()
        d_valid = xgb.DMatrix(X_valid, y_valid,feature_names=feature_subset)
        d_test = xgb.DMatrix(test,feature_names=feature_subset)

    return d_train, d_valid, d_test, X_valid.values, y_valid.values, categorical_cols, feature_subset

In [ ]:
# Get Data
d_train, d_valid, d_test, X_valid, y_valid, categorical_cols, feature_subset = fraud_process(important_cols = important_cols)

In [ ]:
n_estimators = 10000
metric = 'auc'
ESR = 500

params = {
     'colsample_bytree': 0.5317364823333394,
     'eval_metric': metric,
     'importance_type': 'weight',
     'learning_rate': 0.08,
     'max_depth': 9,
     'missing': -9,
#      'num_boost_round': 100,
     'objective': 'binary:logistic',
     'reg_alpha': 0.05,
     'reg_lambda': 0.1749522739788873,
     'seed': 24,
     'subsample': 0.8}

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model = xgb.train(params = params,
                  dtrain = d_train,
                  num_boost_round = n_estimators,
                  evals = watchlist,
                  verbose_eval=500,
                  early_stopping_rounds=ESR)
del d_train

In [ ]:
with timer("Predict / Submission"):
    sample_submission = pd.read_csv(f'{folder_path1}sample_submission.csv', index_col='TransactionID')
    sample_submission['isFraud'] = model.predict(d_test)
    sample_submission.to_csv('xgboost_starter.csv')
    
    del d_test

In [ ]:
gc.collect()

In [ ]:
f, ax = plt.subplots(figsize = [10,20])
xgb.plot_importance(model, max_num_features  = 100, ax = ax)
plt.show

In [ ]:
f_imp = pd.DataFrame.from_dict(model.get_score(), orient='index')\
    .rename(columns = {0: 'importance'})\
    .sort_values(by = "importance", ascending = False)
f_imp.to_csv("Feature_importance.csv")

In [ ]:
sns.distplot(f_imp)
plt.title("Importance Distribution")
plt.show()

print(f_imp.loc[f_imp.importance < 25,'importance'].value_counts())

In [ ]:
%time
t1 = xgb.DMatrix(X_valid,feature_names=feature_subset)
del t1

In [ ]:
permutation_iter = 1

# feature permutation
# ... load data, define score function
def score(X, y):
    val_pred = model.predict(xgb.DMatrix(X,feature_names=feature_subset))
    return roc_auc_score(y, val_pred)
with timer("Permutation Importance"):
    base_score, score_decreases = eli5.permutation_importance.get_score_importances(
                                                        score_func = score,
                                                        X = X_valid,
                                                        y = y_valid,
                                                        n_iter = permutation_iter,
                                                        random_state = seed)
feature_importances = np.mean(score_decreases,axis=0)

# To DF
model_importance = pd.DataFrame()
model_importance['feats'] = feature_subset
model_importance['perm_score'] = np.mean(score_decreases, axis=0)
model_importance['perm_std'] = np.std(score_decreases, axis=0)
model_importance = model_importance.sort_values(by='perm_score', ascending=True)
display(model_importance.head(40))

model_importance.to_csv("Permutation_importance.csv")

In [ ]:
print("Notebook Runtime: %0.2f Hours"%((time.time() - notebookstart)/60/60))